In [1]:
import numpy as np
import pandas as pd

In [5]:
class TreeNode:
  def __init__(self, data,output):
        #split into node at each feature of the data
        self.data = data
        # children of a node are stored as dicticionary with key
        # key is the value upon which each feature is split
        self.children = {}
        # output represents the class with current majority at this instance of the decision tree
        self.output = output
        # index will be used to assign a unique index to each node
        self.index = -1
        
  def add_child(self,feature_value,obj):
      self.children[feature_value] = obj

In [12]:
class DecisionTreeClassifier:
  def __init__(self):
    self.root= None

#To find the number of "Yes" and "No", we define unique_count
  def unique_count(self, Y):
    d={}
    for i in Y:
      if i in d:
        d[i]+=1
      else:
        d[i]= 1
    return d

#For every node, given the unique_count, we define entropy or information required.
  def entropy(self,Y):
    frequency=self.unique_count(Y)
    entropy=0
    for i in range(frequency):
      p=frequency[i]/len(Y)
      entropy+=(-p)*np.log2(p)
    return entropy

#We shall now define the gain ratio, which is ratio of information gain over split information.
  def gain_ratio(self, X,Y, selected_feature):
    info_ori=self.entropy(Y)
    info_f=0 #entropy after splitting upon some particular feature f
    split_info=0
    values = set(X[:,selected_feature])
    #Let us convert X and Y into a dataframe with last column as Y:
    df=pd.DataFrame(X)
    df[df.shape[1]]=Y
    initial_size=df.shape[0]
    for i in values:
      df1=df[df[selected_feature]==i]
      current_size=df1.shape[0]
      info_f+=(current_size)/(initial_size)*self.entropy(df1[df1.shape[1]-1])
      split_info+=(-(current_size)/(initial_size))*np.log2((current_size)/(initial_size))

    if split_info==0:
      return np.inf
    
    info_gain=info_ori-info_f
    gain_ratio=info_gain/split_info
    return gain_ratio

#We will define gini index:
  def gini_index(self,Y):
#probability of a node i is :
    frequency=self.unique_count(Y)
    gini_index=1
    for i in frequency:
      probability=frequency[i]/len(Y)
      gini_index-=probability**2
    return gini_index

#We will now define gini_gain. The higher the value of the gini_gain, the more we decide to go with this feature
  def gini_gain(self,X,Y, selected_feature):
    gini_ori=self.gini_index(Y)
    gini_split_f=0 # defines which split to go with 
    values = set(X[:,selected_feature])
    #Let us convert X and Y into a dataframe with last column as Y:
    df=pd.DataFrame(X)
    df[df.shape[1]]=Y
    initial_size=df.shape[0]
    for i in values:
      df1=df[df[selected_feature]==i]
      current_size=df1.shape[0]
      gini_split_f+=(current_size)/(initial_size)*self.gini_index(df1[df1.shape[1]-1])
    
    gini_gain=gini_ori-gini_split_f
    return gini_gain

  def DecisionTree(self, X, Y, features, classes, metric, level):
    
        # If the node has only 1 class
        if len(set(Y)) == 1:
            print("Level",level)
            output = None
            for i in classes:
                if i in Y:
                    output = i
                    print("Count of",i,"=",len(Y))
                else :
                    print("Count of",i,"=",0)
            if metric == "gain_ratio":
                print("Current Entropy is =  0.0")
            elif metric == "gini_index":
                print("Current Gini Index is =  0.0")

            print("Reached leaf Node")
            print()
            return TreeNode(None,output)

        # If we have run out of features to split on then we return max_count
        if len(features) == 0:
            print("Level",level)
            freq_map = self.unique_count(Y)
            output = None
            max_count = -np.inf
            for i in classes:
                if i not in freq_map:
                    print("Count of",i,"=",0)
                else :
                    if freq_map[i] > max_count :
                        output = i
                        max_count = freq_map[i]
                    print("Count of",i,"=",freq_map[i])

            if metric == "gain_ratio":
                print("Current Entropy  is =",self.entropy(Y))
            elif metric == "gini_index":
                print("Current Gini Index is =",self.gini_index(Y))            

            print("Reached leaf Node")
            print()
            return TreeNode(None,output)

        
        # Finding the best feature to split on
        max_gain = -np.inf
        final_feature = None
        for f in features :
            if metric == "gain_ratio":
                current_gain = self.gain_ratio(X,Y,f)
            elif metric =="gini_index":
                current_gain = self.gini_gain(X,Y,f)

            if current_gain > max_gain:
                max_gain = current_gain
                final_feature = f

        print("Level",level)
        freq_map = self.unique_count(Y)
        output = None
        max_count = -np.inf

        for i in classes:
            if i not in freq_map:
                print("Count of",i,"=",0)
            else :
                if freq_map[i] > max_count :
                    output = i
                    max_count = freq_map[i]
                print("Count of",i,"=",freq_map[i])

        if metric == "gain_ratio" :        
            print("Current Entropy is =",self.entropy(Y))
            print("Splitting on feature  X[",final_feature,"] with gain ratio ",max_gain,sep="")
            print()
        elif metric == "gini_index":
            print("Current Gini Index is =",self.gini_index(Y))
            print("Splitting on feature  X[",final_feature,"] with gini gain ",max_gain,sep="")
            print()

            
        unique_values = set(X[:,final_feature])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y

        current_node = TreeNode(final_feature,output)
        return current_node